In [8]:
#downloading all the dependencies needed

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
#Using the pandas to extract the table of postal codes of the Wikpedia page.
url = "http://www.cidadaorecifense.com.br/saude/unidadessaude/group/99"
df_list = pd.read_html(url) #readind the html file.
df = df_list[0] # defining a dataframe


In [5]:
df.head()

,Nome,Bairro
0,CASA DE SAUDE MARIA LUCINDA (FUNDACAO MANOEL D...,PARNAMIRIM
1,CENTRO HOSPITALAR OSCAR COUTINHO,BOA VISTA
2,CENTRO INTEGRADO DE SAéDE,Engenho do Meio
3,CLINICA PSIQUIATRICA SANTO ANTONIO,MADALENA
4,CLINICA PSIQUIATRICA SANTO ANTONIO DE PADUA,AFOGADOS


In [9]:
#Getting the latitude and longitude values of toronto

address = 'Recife, BR'

geolocator = Nominatim(user_agent="BR_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Recife are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are -8.0641931, -34.8781517.


In [13]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Recife, Brazil'
berlin_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, recife_center))

NameError: name 'google_api_key' is not defined

In [10]:
#Utilizing the Foursquare API to explore the neighborhoods and segment them.

#Define Foursquare Credentials and Version
CLIENT_ID = 'T1F14ZSMPK4DKKGTZASK3MLJU0RMKYNKQXTIYYW3SSTJHY4W' # your Foursquare ID
CLIENT_SECRET = 'EBLGELR3FEVXWRSTVI3EGAFDZFTEY5CG32ARFEIO4V14W1FL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
#Let's explore the first neighborhood in our dataframe.
#Get the neighborhood's name.


Your credentails:
CLIENT_ID: T1F14ZSMPK4DKKGTZASK3MLJU0RMKYNKQXTIYYW3SSTJHY4W
CLIENT_SECRET:EBLGELR3FEVXWRSTVI3EGAFDZFTEY5CG32ARFEIO4V14W1FL


In [11]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

centro_medico = '4bf58dd8d48988d104941735' # 'Root' category for all medic centers

italian_restaurant_categories = ['52e81612bcbc57f1066b7a3b','52e81612bcbc57f1066b7a3c','52e81612bcbc57f1066b7a3a',
                                 '4bf58dd8d48988d178941735','4bf58dd8d48988d177941735','4bf58dd8d48988d194941735',
                                 '522e32fae4b09b556e370f19','4bf58dd8d48988d196941735','58daa1558bbb0b01f18ec1f7',
                                 '56aa371be4b08b9a8d5734ff','4f4531b14b9074f6e4fb0103','52e81612bcbc57f1066b7a39',
                                 '5744ccdfe4b0c0459246b4af','56aa371be4b08b9a8d57351d','56aa371be4b08b9a8d573526']
    

#def is_restaurant(categories, specific_filter=None):
 #   restaurant_words = ['clinica', 'hospital', 'serviço', 'medico']
  #  restaurant = False
   # specific = False
    #for c in categories:
     #   category_name = c[0].lower()
      #  category_id = c[1]
       # for r in restaurant_words:
        #    if r in category_name:
         #       restaurant = True
        #if 'fast food' in category_name:
         #   restaurant = False
        #if not(specific_filter is None) and (category_id in specific_filter):
         #   specific = True
          #  restaurant = True
    #return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

NameError: name 'venues' is not defined